# Data Mining Final Project
##           - Rajendra Prasad Patil

### Glossary:
* Import libraries
* Load dataset
* Analysis on dataset
* Splitting the dataset into labels and features
* Performing normalization on dataset
* Splitting dataset using K fold 
* Running the model
    * SVM Model
    * K Nearest Neighbors
    * Random Forest Classifier
* Output Performance Metrics



### Importing libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

# SVM classifier
from sklearn import svm

# KNN classifier
from sklearn.neighbors import KNeighborsClassifier 

#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

# Import libraries for lstm classification
from keras.layers import Dense, Dropout, LSTM, Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential

# for checking the model accuracy
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

### Loading the dataset

In [2]:
dataset = load_breast_cancer()

In [3]:
input_length = len(dataset['data'][0])

### Preliminary analysis

In [4]:
class_names = dataset['target_names']
print('Target variables  : ', class_names)

(unique, counts) = np.unique(dataset['target'], return_counts=True)

print('Unique values of the target variable', unique)
print('Counts of the target variable :', counts)

Target variables  :  ['malignant' 'benign']
Unique values of the target variable [0 1]
Counts of the target variable : [212 357]


* The dataset is suited for binary classification
* The dataset has no skewed nature

### The data is split into features and labels

In [5]:
X = dataset['data']
y = dataset['target']

### Apply normalization operation for numerical stability

In [6]:
standardizer = StandardScaler()
X = standardizer.fit_transform(X)

# Performance Metrics
Function to calculate all the available performance metrics

In [7]:
performance_metrics = ['True Negative', 'False Positive', 'False Negative', 'True Positivity', 'Sensitivity', 'Specificity', 
                       'Precision', 'Accuracy', 'F1 Score', 'Error Rate', 'Negative Predicted Value', 'False Positve Rate', 
                       'False Discovery Rate', 'False Negative Rate', 'Balanced Accuracy', 'True Skill Statistics', 
                       'Heidke Skill Score']

def compute_performance_metrics(prediction, y_test, df, is_lstm = False):
    
    if is_lstm:
        threshold = 0.80
        for i, each in enumerate(prediction):
            if each[0] > threshold:
                prediction[i] = 1
            else:
                prediction[i] = 0
    
    TN, FP, FN, TP = confusion_matrix(y_test, prediction).ravel()
    
    sensitivity = TP / (TP + FN)
    specificity = TN / (FP + TN)
    precision = TP / (TP + FP)
    accuracy =  (TP+TN) /(TP+FP+TN+FN)
    f1_score = 2 * TP / ((2 * TP) + FP + FN)
    error_rate = (FP + FN) / (TP + FP + FN + TN)
    negative_predicted_value = TN / (TN + FN)
    false_positive_rate = FP / (FP + TN)
    false_discovery_rate = FP / (FP + TP)
    false_negative_rate = FN / (FN + TP)
    balanced_accuracy = 0.5 * ((TP / (TP + FN)) + (TN / (TN + FP)))
    true_skill_statistics = ((TP / (TP + FN)) - (FP / (TN + FP)))
    heidke_skill_score = 2 * ((TP * TN) - (FP * FN)) / (((𝑇𝑃 + 𝐹𝑁) * (𝐹𝑁 + 𝑇𝑁)) +((TP+FP) * (𝐹𝑃 + 𝑇𝑁)))
    
    df = df.append({performance_metrics[0]: TN, performance_metrics[1]: FP, performance_metrics[2]: FN, 
                    performance_metrics[3]: TP, performance_metrics[4]: sensitivity, performance_metrics[5]: specificity, 
                    performance_metrics[6]: precision, performance_metrics[7]: accuracy, performance_metrics[8]: f1_score, 
                    performance_metrics[9]: error_rate, performance_metrics[10]: negative_predicted_value, 
                    performance_metrics[11]: false_positive_rate, performance_metrics[12]: false_discovery_rate, 
                    performance_metrics[13]: false_negative_rate, performance_metrics[14]: 
                    balanced_accuracy, performance_metrics[15]: true_skill_statistics,
                    performance_metrics[16]: heidke_skill_score}, ignore_index=True)
    return df    

# K-fold cross validation

In [8]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=10, shuffle=True, random_state=0)

### Dataframes for performance metrics

In [9]:
svm_metrics_df = pd.DataFrame(columns=performance_metrics)
kn_metrics_df = pd.DataFrame(columns=performance_metrics)
rf_metrics_df = pd.DataFrame(columns=performance_metrics)
lstm_metrics_df = pd.DataFrame(columns=performance_metrics)

## SVM Model

In [10]:
svm_model = svm.SVC()
for train_index, test_index in kfold.split(X):
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]

    # we train the algorithm with training data and training output
    svm_model.fit(X_train, y_train)

    # we pass the testing data to the stored algorithm to predict the outcome
    prediction = svm_model.predict(X_test)

    # print metrics
    svm_metrics_df = compute_performance_metrics(prediction, y_test, svm_metrics_df)

svm_metrics_df.index += 1
svm_metrics_df.loc['Average'] = svm_metrics_df.mean()

In [11]:
svm_metrics_df

,True Negative,False Positive,False Negative,True Positivity,Sensitivity,Specificity,Precision,Accuracy,F1 Score,Error Rate,Negative Predicted Value,False Positve Rate,False Discovery Rate,False Negative Rate,Balanced Accuracy,True Skill Statistics,Heidke Skill Score
1,22.0,0.0,0.0,35.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
2,23.0,2.0,1.0,31.0,0.968750,0.920000,0.939394,0.947368,0.953846,0.052632,0.958333,0.080000,0.060606,0.031250,0.944375,0.888750,0.892655
3,15.0,1.0,1.0,40.0,0.975610,0.937500,0.975610,0.964912,0.975610,0.035088,0.937500,0.062500,0.024390,0.024390,0.956555,0.913110,0.913110
4,20.0,0.0,0.0,37.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
5,17.0,1.0,0.0,39.0,1.000000,0.944444,0.975000,0.982456,0.987342,0.017544,1.000000,0.055556,0.025000,0.000000,0.972222,0.944444,0.958785
6,19.0,3.0,0.0,35.0,1.000000,0.863636,0.921053,0.947368,0.958904,0.052632,1.000000,0.136364,0.078947,0.000000,0.931818,0.863636,0.886076
7,22.0,1.0,1.0,33.0,0.970588,0.956522,0.970588,0.964912,0.970588,0.035088,0.956522,0.043478,0.029412,0.029412,0.963555,0.927110,0.927110
8,23.0,0.0,2.0,32.0,0.941176,1.000000,1.000000,0.964912,0.969697,0.035088,0.920000,0.000000,0.000000,0.058824,0.970588,0.941176,0.928121
9,18.0,0.0,0.0,39.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
10,25.0,0.0,0.0,31.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000


### K-Nearest Neighbors

In [12]:
model = KNeighborsClassifier(n_neighbors=3) # this examines 3 neighbors for putting the data into class

for train_index, test_index in kfold.split(X):
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]

    # we train the algorithm with training data and training output
    model.fit(X_train, y_train)

    # we pass the testing data to the stored algorithm to predict the outcome
    prediction = model.predict(X_test)

    # print metrics
    kn_metrics_df = compute_performance_metrics(prediction, y_test, kn_metrics_df)

kn_metrics_df.index += 1
kn_metrics_df.loc['Average'] = kn_metrics_df.mean()

In [13]:
kn_metrics_df

,True Negative,False Positive,False Negative,True Positivity,Sensitivity,Specificity,Precision,Accuracy,F1 Score,Error Rate,Negative Predicted Value,False Positve Rate,False Discovery Rate,False Negative Rate,Balanced Accuracy,True Skill Statistics,Heidke Skill Score
1,21.0,1.0,0.0,35.0,1.000000,0.954545,0.972222,0.982456,0.985915,0.017544,1.000000,0.045455,0.027778,0.000000,0.977273,0.954545,0.962672
2,21.0,4.0,0.0,32.0,1.000000,0.840000,0.888889,0.929825,0.941176,0.070175,1.000000,0.160000,0.111111,0.000000,0.920000,0.840000,0.854962
3,14.0,2.0,1.0,40.0,0.975610,0.875000,0.952381,0.947368,0.963855,0.052632,0.933333,0.125000,0.047619,0.024390,0.925305,0.850610,0.867133
4,19.0,1.0,0.0,37.0,1.000000,0.950000,0.973684,0.982456,0.986667,0.017544,1.000000,0.050000,0.026316,0.000000,0.975000,0.950000,0.961039
5,17.0,1.0,1.0,38.0,0.974359,0.944444,0.974359,0.964912,0.974359,0.035088,0.944444,0.055556,0.025641,0.025641,0.959402,0.918803,0.918803
6,19.0,3.0,0.0,35.0,1.000000,0.863636,0.921053,0.947368,0.958904,0.052632,1.000000,0.136364,0.078947,0.000000,0.931818,0.863636,0.886076
7,22.0,1.0,0.0,34.0,1.000000,0.956522,0.971429,0.982456,0.985507,0.017544,1.000000,0.043478,0.028571,0.000000,0.978261,0.956522,0.963297
8,23.0,0.0,0.0,34.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
9,18.0,0.0,0.0,39.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
10,23.0,2.0,0.0,31.0,1.000000,0.920000,0.939394,0.964286,0.968750,0.035714,1.000000,0.080000,0.060606,0.000000,0.960000,0.920000,0.927178


### Random Forest Classifier

In [14]:
#Create a Gaussian Classifier
model = RandomForestClassifier(n_estimators=100)

for train_index, test_index in kfold.split(X):
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]

    # we train the algorithm with training data and training output
    model.fit(X_train, y_train)

    # we pass the testing data to the stored algorithm to predict the outcome
    prediction = model.predict(X_test)

    # print metrics
    rf_metrics_df = compute_performance_metrics(prediction, y_test, rf_metrics_df)

rf_metrics_df.index += 1
rf_metrics_df.loc['Average'] = rf_metrics_df.mean()

In [15]:
rf_metrics_df

,True Negative,False Positive,False Negative,True Positivity,Sensitivity,Specificity,Precision,Accuracy,F1 Score,Error Rate,Negative Predicted Value,False Positve Rate,False Discovery Rate,False Negative Rate,Balanced Accuracy,True Skill Statistics,Heidke Skill Score
1,22.0,0.0,3.0,32.0,0.914286,1.000000,1.000000,0.947368,0.955224,0.052632,0.880000,0.000000,0.000000,0.085714,0.957143,0.914286,0.891704
2,24.0,1.0,0.0,32.0,1.000000,0.960000,0.969697,0.982456,0.984615,0.017544,1.000000,0.040000,0.030303,0.000000,0.980000,0.960000,0.964218
3,15.0,1.0,2.0,39.0,0.951220,0.937500,0.975000,0.947368,0.962963,0.052632,0.882353,0.062500,0.025000,0.048780,0.944360,0.888720,0.872102
4,19.0,1.0,0.0,37.0,1.000000,0.950000,0.973684,0.982456,0.986667,0.017544,1.000000,0.050000,0.026316,0.000000,0.975000,0.950000,0.961039
5,17.0,1.0,2.0,37.0,0.948718,0.944444,0.973684,0.947368,0.961039,0.052632,0.894737,0.055556,0.026316,0.051282,0.946581,0.893162,0.880000
6,18.0,4.0,2.0,33.0,0.942857,0.818182,0.891892,0.894737,0.916667,0.105263,0.900000,0.181818,0.108108,0.057143,0.880519,0.761039,0.774108
7,22.0,1.0,0.0,34.0,1.000000,0.956522,0.971429,0.982456,0.985507,0.017544,1.000000,0.043478,0.028571,0.000000,0.978261,0.956522,0.963297
8,20.0,3.0,1.0,33.0,0.970588,0.869565,0.916667,0.929825,0.942857,0.070175,0.952381,0.130435,0.083333,0.029412,0.920077,0.840153,0.852140
9,18.0,0.0,0.0,39.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
10,25.0,0.0,0.0,31.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000


# LSTM classifier

In [16]:
model = Sequential()
model.add(LSTM(20, input_shape=(input_length, 1)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

for train_index, test_index in kfold.split(X):
    print('*'*100)
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]

    # we train the algorithm with training data and training output
    model.fit(X_train, y_train, batch_size=8, epochs=10, validation_data=(X_test, y_test), verbose = 1)

    # we pass the testing data to the stored algorithm to predict the outcome
    prediction = model.predict(X_test)
    
    # print metrics
    lstm_metrics_df = compute_performance_metrics(prediction, y_test, lstm_metrics_df, is_lstm=True)

lstm_metrics_df.index += 1
lstm_metrics_df.loc['Average'] = lstm_metrics_df.mean()

****************************************************************************************************
Epoch 1/10
64/64 [==============================] - 4s 23ms/step - loss: 0.5242 - accuracy: 0.8652 - val_loss: 0.3321 - val_accuracy: 0.8947
Epoch 2/10
64/64 [==============================] - 1s 12ms/step - loss: 0.3274 - accuracy: 0.8965 - val_loss: 0.2557 - val_accuracy: 0.9123
Epoch 3/10
64/64 [==============================] - 1s 12ms/step - loss: 0.3026 - accuracy: 0.8965 - val_loss: 0.2491 - val_accuracy: 0.9123
Epoch 4/10
64/64 [==============================] - 1s 12ms/step - loss: 0.2851 - accuracy: 0.9062 - val_loss: 0.2335 - val_accuracy: 0.9298
Epoch 5/10
64/64 [==============================] - 1s 12ms/step - loss: 0.2854 - accuracy: 0.9102 - val_loss: 0.2548 - val_accuracy: 0.9123
Epoch 6/10
64/64 [==============================] - 1s 12ms/step - loss: 0.2841 - accuracy: 0.9102 - val_loss: 0.2355 - val_accuracy: 0.9298
Epoch 7/10
64/64 [==============================] - 1

Epoch 5/10
64/64 [==============================] - 1s 12ms/step - loss: 0.1094 - accuracy: 0.9688 - val_loss: 0.2035 - val_accuracy: 0.9298
Epoch 6/10
64/64 [==============================] - 1s 12ms/step - loss: 0.1047 - accuracy: 0.9668 - val_loss: 0.2173 - val_accuracy: 0.9123
Epoch 7/10
64/64 [==============================] - 1s 12ms/step - loss: 0.1214 - accuracy: 0.9531 - val_loss: 0.2254 - val_accuracy: 0.9123
Epoch 8/10
64/64 [==============================] - 1s 12ms/step - loss: 0.1052 - accuracy: 0.9629 - val_loss: 0.2213 - val_accuracy: 0.9298
Epoch 9/10
64/64 [==============================] - 1s 12ms/step - loss: 0.1044 - accuracy: 0.9707 - val_loss: 0.2235 - val_accuracy: 0.9298
Epoch 10/10
64/64 [==============================] - 1s 12ms/step - loss: 0.1110 - accuracy: 0.9609 - val_loss: 0.2265 - val_accuracy: 0.9298
****************************************************************************************************
Epoch 1/10
64/64 [==============================] - 

In [17]:
lstm_metrics_df

,True Negative,False Positive,False Negative,True Positivity,Sensitivity,Specificity,Precision,Accuracy,F1 Score,Error Rate,Negative Predicted Value,False Positve Rate,False Discovery Rate,False Negative Rate,Balanced Accuracy,True Skill Statistics,Heidke Skill Score
1,21.0,1.0,3.0,32.0,0.914286,0.954545,0.969697,0.929825,0.941176,0.070175,0.875000,0.045455,0.030303,0.085714,0.934416,0.868831,0.854406
2,22.0,3.0,2.0,30.0,0.937500,0.880000,0.909091,0.912281,0.923077,0.087719,0.916667,0.120000,0.090909,0.062500,0.908750,0.817500,0.821092
3,15.0,1.0,6.0,35.0,0.853659,0.937500,0.972222,0.877193,0.909091,0.122807,0.714286,0.062500,0.027778,0.146341,0.895579,0.791159,0.722338
4,19.0,1.0,0.0,37.0,1.000000,0.950000,0.973684,0.982456,0.986667,0.017544,1.000000,0.050000,0.026316,0.000000,0.975000,0.950000,0.961039
5,17.0,1.0,3.0,36.0,0.923077,0.944444,0.972973,0.929825,0.947368,0.070175,0.850000,0.055556,0.027027,0.076923,0.933761,0.867521,0.842324
6,20.0,2.0,2.0,33.0,0.942857,0.909091,0.942857,0.929825,0.942857,0.070175,0.909091,0.090909,0.057143,0.057143,0.925974,0.851948,0.851948
7,22.0,1.0,3.0,31.0,0.911765,0.956522,0.968750,0.929825,0.939394,0.070175,0.880000,0.043478,0.031250,0.088235,0.934143,0.868286,0.856242
8,23.0,0.0,3.0,31.0,0.911765,1.000000,1.000000,0.947368,0.953846,0.052632,0.884615,0.000000,0.000000,0.088235,0.955882,0.911765,0.892924
9,18.0,0.0,0.0,39.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
10,25.0,0.0,2.0,29.0,0.935484,1.000000,1.000000,0.964286,0.966667,0.035714,0.925926,0.000000,0.000000,0.064516,0.967742,0.935484,0.928297


### Observation
* I consider balanced accuracy to be the optimal metric to find the best model.
* The case being, SVM is the model which is giving the highest balanced accuracy.

### Why is SVM performing better?
* SVM doesn't get affected by outliers
* It does not suffer from overfitting
* It is more efficient than other ML algorithms listed here